In [11]:
import requests
import pandas as pd
import numpy as np
import io
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelog, boxscoreadvancedv2
import matplotlib.pyplot as plt

In [12]:
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [13]:
team_dict = pd.DataFrame(teams.get_teams())
abbr_to_id = team_dict.set_index('abbreviation').to_dict()['id']
boston_id = abbr_to_id['BOS']

gamelog = teamgamelog.TeamGameLog(team_id=boston_id, season='2021-22', date_from_nullable='01/01/2022', headers=headers)
games = gamelog.get_data_frames()[0]
opponents = []
for value in games['MATCHUP'].values:
    opponents.append(abbr_to_id[value[-3:]])

bos_games = pd.DataFrame()
bos_games['OPP_ID'] = opponents
bos_games['Game_ID'] = games['Game_ID'].unique()

def get_box_advanced(game_id):
    return boxscoreadvancedv2.BoxScoreAdvancedV2(game_id, headers=headers).team_stats.get_data_frame()

In [14]:
def get_opp_winpct(team_id, game_id):
    
    gamelog = teamgamelog.TeamGameLog(team_id=team_id, season='2021-22', headers=headers)
    games = gamelog.get_data_frames()[0]
    game = games.loc[games['Game_ID'] == game_id]

    return game['W_PCT'].values[0]

In [15]:
box_score_agg = pd.DataFrame()
for game_id, opp_id in zip(bos_games['Game_ID'], bos_games['OPP_ID']):
    box = get_box_advanced(game_id)
    box = box[box['TEAM_NAME'] == 'Celtics']
    box['OPP_OVER_500'] = True if (get_opp_winpct(opp_id, game_id) >= .5) else False
    box_score_agg = pd.concat([box_score_agg, box], axis=0, ignore_index=1)

bos_games = pd.concat([bos_games, box_score_agg], axis=1)

In [16]:
df_bos_agg = bos_games.groupby('OPP_OVER_500').agg(
    
    GAMES = ('TEAM_NAME', "count"),
    OFF_EFF = ('OFF_RATING', "mean"),
    DEF_EFF = ('DEF_RATING', "mean"),

)

In [17]:
df_bos_agg

,GAMES,OFF_EFF,DEF_EFF
OPP_OVER_500,,,
False,22,113.118182,103.090909
True,10,120.350000,104.750000


In [18]:
bos_games

,OPP_ID,Game_ID,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,...,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,OPP_OVER_500
0,1610612765,0022100997,0022100997,1610612738,Celtics,BOS,Boston,240:00,114.9,117.5,...,0.500,0.552,1.0,0.200,97.88,96.50,80.42,97,0.568,False
1,1610612766,0022100981,0022100981,1610612738,Celtics,BOS,Boston,240:00,122.1,122.3,...,0.559,0.580,1.0,0.198,96.24,94.50,78.75,94,0.588,False
2,1610612751,0022100960,0022100960,1610612738,Celtics,BOS,Boston,240:00,126.4,126.0,...,0.584,0.632,1.0,0.201,100.76,100.50,83.75,100,0.526,False
3,1610612763,0022100940,0022100940,1610612738,Celtics,BOS,Boston,240:00,126.8,129.0,...,0.607,0.628,1.0,0.197,94.66,93.00,77.50,93,0.575,True
4,1610612737,0022100927,0022100927,1610612738,Celtics,BOS,Boston,240:00,113.4,113.8,...,0.536,0.586,1.0,0.196,96.46,94.50,78.75,94,0.581,False
5,1610612754,0022100914,0022100914,1610612738,Celtics,BOS,Boston,240:00,109.4,108.1,...,0.495,0.536,1.0,0.200,100.30,98.50,82.08,99,0.441,False
6,1610612765,0022100905,0022100905,1610612738,Celtics,BOS,Boston,240:00,116.2,118.9,...,0.534,0.581,1.0,0.198,98.96,95.50,79.58,95,0.577,False
7,1610612751,0022100890,0022100890,1610612738,Celtics,BOS,Boston,240:00,125.4,125.2,...,0.641,0.666,1.0,0.198,106.10,104.00,86.67,103,0.605,True
8,1610612765,0022100875,0022100875,1610612738,Celtics,BOS,Boston,240:00,111.1,112.1,...,0.630,0.654,1.0,0.196,98.82,99.00,82.50,99,0.523,False
9,1610612755,0022100869,0022100869,1610612738,Celtics,BOS,Boston,240:00,141.1,145.2,...,0.713,0.736,1.0,0.195,94.92,93.50,77.92,93,0.713,True
